###### https://www.youtube.com/watch?v=i_LwzRVP7bg&list=PLWKjhJtqVAblStefaz_YOVpDWqcRScc2s&index=1
###### https://colab.research.google.com/drive/1m3oQ9b0oYOT-DXEy0JCdgWPLGllHMb4V?usp=sharing#scrollTo=17bmL-NpKUhE
###### https://colab.research.google.com/drive/16w3TDn_tAku17mum98EWTmjaLHAJcsk0?usp=sharing#scrollTo=-Vohv6aAzZFK

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# from sklearn.preprocessing import StandardScaler
# from imblearn.over_sampling import RandomOverSampler

## data import

In [24]:
cols = ["fLength", "fWidth", "fSize", "fConc", "fConc1", "fAsym", "fM3Long", "fM3Trans", "fAlpha", "fDist", "class"]
df=pd.read_csv("magic04.data",names=cols)


In [ ]:
df.head()

In [26]:
df["class"] = (df["class"] == "g").astype(int)

In [ ]:
df["class"].value_counts()[:]

In [ ]:
df.info()

In [ ]:
for label in cols[:-1]:
  plt.hist(df[df["class"]==1][label], color='blue', label='gamma', alpha=0.7, density=True) # density=True for normalize data
  plt.hist(df[df["class"]==0][label], color='red', label='hadron', alpha=0.7, density=True)
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  plt.show()

### Train, validation, test datasets

In [21]:
# Randomly shuffle the DataFrame and split it into training (60%), validation (20%), and test sets (20%)
# random_state=42 ensures reproducibility, so the shuffling is the same every time you run the code
train, valid, test = np.split(df.sample(frac=1, random_state=42), [int(0.6*len(df)), int(0.8*len(df))])